In [3]:
import requests
from urllib.parse import urlencode, urlparse, parse_qsl

In [4]:
api_key = ""
with open('api_key.txt') as f:
    api_key = f.readline()
    f.close

##### Google Maps Client API

In [5]:
class GoogleMapsClient(object):
    lat = None
    lng = None
    data_type = 'json'
    location_query = None
    api_key = None
    
    def __init__(self, api_key=None, address_or_postal_code=None, *args, **kwargs):
        super().__init__(*args, **kwargs)
        if api_key == None:
            raise Exception("API key is required")
        self.api_key = api_key
        self.location_query = address_or_postal_code
        if self.location_query != None:
            self.extract_lat_lng()
    
    def extract_lat_lng(self, location=None):
        loc_query = self.location_query
        if location != None:
            loc_query = location
        endpoint = f"https://maps.googleapis.com/maps/api/geocode/{self.data_type}"
        params = {"address": loc_query, "key": self.api_key}
        url_params = urlencode(params)
        url = f"{endpoint}?{url_params}"
        r = requests.get(url)
        if r.status_code not in range(200, 299): 
            return {}
        latlng = {}
        try:
            latlng = r.json()['results'][0]['geometry']['location']
        except:
            pass
        lat,lng = latlng.get("lat"), latlng.get("lng")
        self.lat = lat
        self.lng = lng
        return lat, lng
    
    def search(self, keyword="Mexican food", radius=5000, location=None):
        lat, lng = self.lat, self.lng
        if location != None:
            lat, lng = self.extract_lat_lng(location=location)
        endpoint = f"https://maps.googleapis.com/maps/api/place/nearbysearch/{self.data_type}"
        params = {
            "key": self.api_key,
            "location": f"{lat},{lng}",
            "radius": radius,
            "keyword": keyword
        }
        params_encoded = urlencode(params)
        places_url = f"{endpoint}?{params_encoded}"
        r = requests.get(places_url)
        # print(places_url, r.text)
        if r.status_code not in range(200, 299):
            return {}
        return r.json()
    
    def detail(self, place_id="ChIJlXOKcDC3j4ARzal-5j-p-FY", fields=["name", "rating", "formatted_phone_number", "formatted_address"]):
        detail_base_endpoint = f"https://maps.googleapis.com/maps/api/place/details/{self.data_type}"
        detail_params = {
            "place_id": f"{place_id}",
            "fields" : ",".join(fields),
            "key": self.api_key
        }
        detail_params_encoded = urlencode(detail_params)
        detail_url = f"{detail_base_endpoint}?{detail_params_encoded}"
        r = requests.get(detail_url)
        if r.status_code not in range(200, 299):
            return {}
        return r.json()

In [7]:
client = GoogleMapsClient(api_key=api_key, address_or_postal_code='rabat')
print(client.lat, client.lng)

33.9715904 -6.8498129


Search all the possible libraries in a 5000m radius

In [23]:
librairies = client.search("Librairie", radius=5000)

In [24]:
librairies

{'html_attributions': [],
 'next_page_token': 'Aap_uECISSuH7WA3hKCddkqcTBEzPIvFkU6EuKRJdXW0XvBdr-3JJrA1bCyh3_5AjnudkWkzEALUgaOzJkvJKYsE41jqwtsemq4y9cRR8vc7yCp6oB2rtFspODgRq54apEBRBB2CumMpEub5V5HDqBn6TgFmAhOkNYtPn_vsDof87y9HfJXiVYqri5BJSMYCJKYyn4VeLYESlfYINxrE8q0D3aPEo2lI-xhCA5u8xiEs6lhyvzmgEqgTHEIUYtObnNXu919LDijV_qVc-F-1WI4-lz5-k4O_hDV2oSCDeNY6cE5SAi9nhvGfYbkXR0SRfIpCK1d5ziiM1j9MdacXrrIk-jjzpGOLAK9CJdHLRkANwj9wPYdVEOdbAcSaHTf4DLeKDW2Daj9JwXCJEEqjAhk2vdIlMIU-CU_adsKVp2a7EkEFPuxfFdL3z0WJEOY',
 'results': [{'business_status': 'OPERATIONAL',
   'geometry': {'location': {'lat': 33.9934298, 'lng': -6.8461155},
    'viewport': {'northeast': {'lat': 33.99464762989273,
      'lng': -6.844747770107278},
     'southwest': {'lat': 33.99194797010728, 'lng': -6.847447429892722}}},
   'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/shopping-71.png',
   'name': 'Librairie Papeterie Basta',
   'opening_hours': {'open_now': True},
   'photos': [{'height': 864,
     'html_attributio

In [25]:
for i in librairie['results']:
    print(i['name'])

Librairie Papeterie Basta
Librairie Et Journaux
LIVRE OUVERT
Genuisland
Bookstore stationery Agdal
Librairie Lebeau-Livre
librairie papeterie l'Encyclopédie
مكتبة ووراقة الروائع Librairie Papeterie Les Merveilles
Librairie Papeterie infomanzah
Librairie al atlas
Papetrie librairie madagascar
Livres & loisirs
Librairie Papeterie Al Majd
librairie la reference
Librairie Internationale
Librairie Almaarif Annexe
LIBRAIRIE & PAPETERIE ADAMI
amigos.librairie
Educalibrary
Librairie Imad | مكتبة عماد
